In [1]:
# TSP without Scott and Morril Traditions (Using Kennedy Traditions)

# import packages
import pandas as pd
import numpy as np
import csv
import pyomo.environ as pyEnv

# get distance matrix
b = np.array([[0, 220, 210, 250, 89, 180, 450, 200, 500, 500, 600, 800, 550, 550, 900, 850, 450, 400, 450],
 [220, 0, 210, 260, 300 ,400, 700, 260, 550, 500, 650, 800, 550, 500, 850, 800, 350, 180, 260],
 [210, 210, 0, 110, 210, 260, 550, 110, 400, 350, 500, 600, 350, 350, 700, 650, 280, 280, 450],
 [250, 260, 110, 0, 220, 180, 450, 10, 300, 270, 400, 550, 350, 400, 700, 650, 260, 350, 500],
 [89, 300, 210, 220 ,0, 95, 400 ,110, 400, 400, 500, 700, 500, 600, 850, 850 ,450, 450, 550], 
 [180, 400, 260, 180, 95, 0 ,300, 67 ,350, 350, 450, 650, 450, 550, 800, 800, 450, 500, 600],
 [450, 700, 550 ,450, 400, 300, 0, 270, 350, 500, 500 ,700, 650, 800, 900, 1000, 750, 800, 900],
 [200, 260, 110 ,10 ,110, 67, 270, 0, 200, 240, 350, 500, 260, 260, 600, 500, 140, 250, 450],
 [500, 550, 400, 300 ,400, 350, 350, 200, 0, 180, 160, 350, 300, 500, 550, 650, 500 ,650, 800],
 [500, 500, 350, 270 ,400, 350, 500, 240, 180, 0 ,170, 350, 180, 350, 500, 550, 400, 500, 700],
 [600, 650, 500, 400, 500, 450, 500, 350, 160, 170, 0, 200, 280, 450, 550, 600, 550, 650, 850],
 [800, 800, 600, 550, 700, 650, 700, 500, 350, 350, 200, 0, 300, 400, 500, 550, 600, 700, 1000],
 [550, 550, 350, 350, 500, 450, 650, 260, 300, 180, 280, 300, 0, 170, 350, 450, 300, 400, 700],
 [550, 500, 350 ,400, 600, 550, 800, 260, 500, 350, 450, 400, 170, 0, 400, 270, 200, 400, 650],
 [900, 850, 700, 700, 850, 800, 900, 600, 550, 500, 550, 500, 350 ,400, 0, 130, 550, 750, 1100],
 [850, 800, 650, 650, 850, 800, 1000, 500, 650, 550, 600, 550, 450, 270, 130, 0, 450, 600, 1000],
 [450, 350, 280, 260 ,450 ,450, 750, 140, 500, 400, 550, 600, 300, 200 ,550, 450, 0, 180, 500],
 [400, 180 ,280, 350, 450, 500, 800, 250, 650, 500, 650, 700, 400, 400, 750, 600 ,180, 0, 350],
 [450, 260 ,450 ,500, 550, 600,900, 450, 800 ,700 ,850, 1000, 700, 650, 1100, 1000, 500, 350, 0]])
n = len(b)


# Model
# ConcreteModel() creates the model
model = pyEnv.ConcreteModel()

# Indexes for the buildings
# RangeSet(n) creates an index from 1 to n
model.M = pyEnv.RangeSet(n)
model.N = pyEnv.RangeSet(n)
model.M.pprint()

# Index for dummy variable U
# RangeSet(2,n) creates an index from 2 to n
model.U = pyEnv.RangeSet(2,n)

# Decision variables xij
# function creates n binary decision variables of size m
model.x = pyEnv.Var(model.N, model.M, within=pyEnv.Binary)

# Dummary variable ui
# creates n non-negative integer decision variables that can only assume values between 0 and n-1
model.u = pyEnv.Var(model.N, within=pyEnv.NonNegativeIntegers, bounds=(0, n-1))

# distance matrix
# provides an nxm parameter to the model using a lambda function
model.d = pyEnv.Param(model.N, model.M, initialize=lambda model, i, j:b[i-1][j-1])

# declare objective function
def obj_func(model): 
    return sum(model.x[i,j]*model.d[i,j] for i in model.N for j in model.M)
model.objective = pyEnv.Objective(rule=obj_func, sense=pyEnv.minimize)

# constraint
# creates m constraints defined by rule_const1
# ensures that each building is arrived at from exactly one other building
def rule_const1(model,M):
    return sum(model.x[i,M] for i in model.N if i!=M ) == 1

model.const1 = pyEnv.Constraint(model.M,rule=rule_const1)

# constraint
# creates n constraints defined by rule_const2
# ensures that each building is a departure to exactly one building
def rule_const2(model,N):
    return sum(model.x[N,j] for j in model.M if j!=N) == 1

model.rest2 = pyEnv.Constraint(model.N,rule=rule_const2)

# constraint
# creates txn constraints defined by rule_const3
# subtour elimination
def rule_const3(model,i,j):
    if i!=j: 
       return model.u[i] - model.u[j] + model.x[i,j] * n <= (n-1)
    else:
        #Yeah, this else doesn't say anything
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = pyEnv.Constraint(model.U,model.N,rule=rule_const3)

# no consecutive libraries
def rule_const6(model):
    return model.x[11,14] == 0
model.rest6 = pyEnv.Constraint(rule=rule_const6)

def rule_const7(model):
    return model.x[14,11] == 0
model.rest7 = pyEnv.Constraint(rule=rule_const7)

# Buildings with time constraint go last
def rule_const10(model):
    return model.x[3,4] == 1
model.rest10 = pyEnv.Constraint(rule=rule_const10)

def rule_const11(model):
    return model.x[4,5] == 1
model.rest11 = pyEnv.Constraint(rule=rule_const11)

def rule_const12(model):
    return model.x[5,6] == 1
model.rest12 = pyEnv.Constraint(rule=rule_const12)

def rule_const13(model):
    return model.x[6,1] == 1
model.rest13 = pyEnv.Constraint(rule=rule_const13)

# Specify only 19 locations to be visited
def rule_const17(model):
    return sum(model.x[i,j] for i in model.N for j in model.M) == 19
model.constraint17 = pyEnv.Constraint(rule=rule_const17)

opt = pyEnv.SolverFactory("gurobi", solver_io="python") 
results = opt.solve(model)
results.write()

# prints results
print(results)

# See decision variables
List = list(model.x.keys())
for i in List:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())

M : Dimen=1, Size=19, Bounds=(1, 19)
    Key  : Finite : Members
    None :   True :  [1:19]
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4145.0
  Upper bound: 4145.0
  Number of objectives: 1
  Number of constraints: 387
  Number of variables: 380
  Number of binary variables: 361
  Number of integer variables: 380
  Number of continuous variables: -361
  Number of nonzeros: 2023
  Sense: 1
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 9.52
  Status: ok
  Wallclock time: 0.087860107421875
  Termination condition: optimal

In [2]:
# TSP without Kennedy and Morril Traditions (Using Scott Traditions)

# import packages
import pandas as pd
import numpy as np
import csv
import pyomo.environ as pyEnv

# get distance matrix
b = np.array([[0, 220, 210, 250, 89, 180, 450, 200, 500, 500, 600, 800, 550, 550, 900, 850, 450, 400, 850],
 [220, 0, 210, 260, 300 ,400, 700, 260, 550, 500, 650, 800, 550, 500, 850, 800, 350, 180, 900],
 [210, 210, 0, 110, 210, 260, 550, 110, 400, 350, 500, 600, 350, 350, 700, 650, 280, 280, 750],
 [250, 260, 110, 0, 220, 180, 450, 10, 300, 270, 400, 550, 350, 400, 700, 650, 260, 350, 650],
 [89, 300, 210, 220 ,0, 95, 400 ,110, 400, 400, 500, 700, 500, 600, 850, 850 ,450, 450, 750],
 [180, 400, 260, 180, 95, 0 ,300, 67 ,350, 350, 450, 650, 450, 550, 800, 800, 450, 500, 700],
 [450, 700, 550 ,450, 400, 300, 0, 270, 350, 500, 500 ,700, 650, 800, 900, 1000, 750, 800, 700],
 [200, 260, 110 ,10 ,110, 67, 270, 0, 200, 240, 350, 500, 260, 260, 600, 500, 140, 250, 650],
 [500, 550, 400, 300 ,400, 350, 350, 200, 0, 180, 160, 350, 300, 500, 550, 650, 500 ,650, 400],
 [500, 500, 350, 270 ,400, 350, 500, 240, 180, 0 ,170, 350, 180, 350, 500, 550, 400, 500, 450],
 [600, 650, 500, 400, 500, 450, 500, 350, 160, 170, 0, 200, 280, 450, 550, 600, 550, 650, 270],
 [800, 800, 600, 550, 700, 650, 700, 500, 350, 350, 200, 0, 300, 400, 500, 550, 600, 700, 350],
 [550, 550, 350, 350, 500, 450, 650, 260, 300, 180, 280, 300, 0, 170, 350, 450, 300, 400, 550],
 [550, 500, 350 ,400, 600, 550, 800, 260, 500, 350, 450, 400, 170, 0, 400, 270, 200, 400, 700],
 [900, 850, 700, 700, 850, 800, 900, 600, 550, 500, 550, 500, 350 ,400, 0, 130, 550, 750, 700],
 [850, 800, 650, 650, 850, 800, 1000, 500, 650, 550, 600, 550, 450, 270, 130, 0, 450, 600, 850],
 [450, 350, 280, 260 ,450 ,450, 750, 140, 500, 400, 550, 600, 300, 200 ,550, 450, 0, 180, 800],
 [400, 180 ,280, 350, 450, 500, 800, 250, 650, 500, 650, 700, 400, 400, 750, 600 ,180, 0, 950],
 [850, 900 ,750, 650, 750 ,700, 700, 650, 400, 450, 270, 350, 550, 700, 700, 850, 800, 950, 0]])
n = len(b)


# Model
# ConcreteModel() creates the model
model = pyEnv.ConcreteModel()

# Indexes for the buildings
# RangeSet(n) creates an index from 1 to n
model.M = pyEnv.RangeSet(n)
model.N = pyEnv.RangeSet(n)
model.M.pprint()

# Index for dummy variable U
# RangeSet(2,n) creates an index from 2 to n
model.U = pyEnv.RangeSet(2,n)

# Decision variables xij
# function creates n binary decision variables of size m
model.x = pyEnv.Var(model.N, model.M, within=pyEnv.Binary)

# Dummary variable ui
# creates n non-negative integer decision variables that can only assume values between 0 and n-1
model.u = pyEnv.Var(model.N, within=pyEnv.NonNegativeIntegers, bounds=(0, n-1))

# distance matrix
# provides an nxm parameter to the model using a lambda function
model.d = pyEnv.Param(model.N, model.M, initialize=lambda model, i, j:b[i-1][j-1])

# declare objective function
def obj_func(model): 
    return sum(model.x[i,j]*model.d[i,j] for i in model.N for j in model.M)
model.objective = pyEnv.Objective(rule=obj_func, sense=pyEnv.minimize)

# constraint
# creates m constraints defined by rule_const1
# ensures that each building is arrived at from exactly one other building
def rule_const1(model,M):
    return sum(model.x[i,M] for i in model.N if i!=M ) == 1

model.const1 = pyEnv.Constraint(model.M,rule=rule_const1)

# constraint
# creates n constraints defined by rule_const2
# ensures that each building is a departure to exactly one building
def rule_const2(model,N):
    return sum(model.x[N,j] for j in model.M if j!=N) == 1

model.rest2 = pyEnv.Constraint(model.N,rule=rule_const2)

# constraint
# creates txn constraints defined by rule_const3
# subtour elimination
def rule_const3(model,i,j):
    if i!=j: 
       return model.u[i] - model.u[j] + model.x[i,j] * n <= (n-1)
    else:
        #Yeah, this else doesn't say anything
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = pyEnv.Constraint(model.U,model.N,rule=rule_const3)

# no consecutive libraries
def rule_const6(model):
    return model.x[11,14] == 0
model.rest6 = pyEnv.Constraint(rule=rule_const6)

def rule_const7(model):
    return model.x[14,11] == 0
model.rest7 = pyEnv.Constraint(rule=rule_const7)

# Buildings with time constraint go last
def rule_const10(model):
    return model.x[3,4] == 1
model.rest10 = pyEnv.Constraint(rule=rule_const10)

def rule_const11(model):
    return model.x[4,5] == 1
model.rest11 = pyEnv.Constraint(rule=rule_const11)

def rule_const12(model):
    return model.x[5,6] == 1
model.rest12 = pyEnv.Constraint(rule=rule_const12)

def rule_const13(model):
    return model.x[6,1] == 1
model.rest13 = pyEnv.Constraint(rule=rule_const13)

# Specify only 19 locations to be visited
def rule_const17(model):
    return sum(model.x[i,j] for i in model.N for j in model.M) == 19
model.constraint17 = pyEnv.Constraint(rule=rule_const17)

opt = pyEnv.SolverFactory("gurobi", solver_io="python") 
results = opt.solve(model)
results.write()

# prints results
print(results)

# See decision variables
List = list(model.x.keys())
for i in List:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())

M : Dimen=1, Size=19, Bounds=(1, 19)
    Key  : Finite : Members
    None :   True :  [1:19]
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4135.0
  Upper bound: 4135.0
  Number of objectives: 1
  Number of constraints: 387
  Number of variables: 380
  Number of binary variables: 361
  Number of integer variables: 380
  Number of continuous variables: -361
  Number of nonzeros: 2023
  Sense: 1
  Number of solutions: 4
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 9.52
  Status: ok
  Wallclock time: 0.04888916015625
  Termination condition: optimal


In [3]:
# TSP without Scott and Kennedy Traditions (Using Morril Traditions)

# import packages
import pandas as pd
import numpy as np
import csv
import pyomo.environ as pyEnv

# get distance matrix
b = np.array([[0, 220, 210, 250, 89, 180, 450, 200, 500, 500, 600, 800, 550, 550, 900, 850, 450, 400, 1300],
 [220, 0, 210, 260, 300 ,400, 700, 260, 550, 500, 650, 800, 550, 500, 850, 800, 350, 180, 1300],
 [210, 210, 0, 110, 210, 260, 550, 110, 400, 350, 500, 600, 350, 350, 700, 650, 280, 280, 1100],
 [250, 260, 110, 0, 220, 180, 450, 10, 300, 270, 400, 550, 350, 400, 700, 650, 260, 350, 1100],
 [89, 300, 210, 220 ,0, 95, 400 ,110, 400, 400, 500, 700, 500, 600, 850, 850 ,450, 450, 1300],
 [180, 400, 260, 180, 95, 0 ,300, 67 ,350, 350, 450, 650, 450, 550, 800, 800, 450, 500, 1200],
 [450, 700, 550 ,450, 400, 300, 0, 270, 350, 500, 500 ,700, 650, 800, 900, 1000, 750, 800, 1300],
 [200, 260, 110 ,10 ,110, 67, 270, 0, 200, 240, 350, 500, 260, 260, 600, 500, 140, 250, 1000],
 [500, 550, 400, 300 ,400, 350, 350, 200, 0, 180, 160, 350, 300, 500, 550, 650, 500 ,650, 1000],
 [500, 500, 350, 270 ,400, 350, 500, 240, 180, 0 ,170, 350, 180, 350, 500, 550, 400, 500, 900],
 [600, 650, 500, 400, 500, 450, 500, 350, 160, 170, 0, 200, 280, 450, 550, 600, 550, 650, 950],
 [800, 800, 600, 550, 700, 650, 700, 500, 350, 350, 200, 0, 300, 400, 500, 550, 600, 700, 900],
 [550, 550, 350, 350, 500, 450, 650, 260, 300, 180, 280, 300, 0, 170, 350, 450, 300, 400, 800],
 [550, 500, 350 ,400, 600, 550, 800, 260, 500, 350, 450, 400, 170, 0, 400, 270, 200, 400, 750],
 [900, 850, 700, 700, 850, 800, 900, 600, 550, 500, 550, 500, 350 ,400, 0, 130, 550, 750, 140],
 [850, 800, 650, 650, 850, 800, 1000, 500, 650, 550, 600, 550, 450, 270, 130, 0, 450, 600, 500],
 [450, 350, 280, 260 ,450 ,450, 750, 140, 500, 400, 550, 600, 300, 200 ,550, 450, 0, 180, 900],
 [400, 180 ,280, 350, 450, 500, 800, 250, 650, 500, 650, 700, 400, 400, 750, 600 ,180, 0, 1100],
 [1300, 1300, 1100, 1100, 1300, 1200, 1300, 1000, 1000, 900, 950, 900, 800, 750, 140, 500, 900, 1100, 0]])
n = len(b)


# Model
# ConcreteModel() creates the model
model = pyEnv.ConcreteModel()

# Indexes for the buildings
# RangeSet(n) creates an index from 1 to n
model.M = pyEnv.RangeSet(n)
model.N = pyEnv.RangeSet(n)

# Index for dummy variable U
# RangeSet(2,n) creates an index from 2 to n
model.U = pyEnv.RangeSet(2,n)

# Decision variables xij
# function creates n binary decision variables of size m
model.x = pyEnv.Var(model.N, model.M, within=pyEnv.Binary)

# Dummary variable ui
# creates n non-negative integer decision variables that can only assume values between 0 and n-1
model.u = pyEnv.Var(model.N, within=pyEnv.NonNegativeIntegers, bounds=(0, n-1))

# distance matrix
# provides an nxm parameter to the model using a lambda function
model.d = pyEnv.Param(model.N, model.M, initialize=lambda model, i, j:b[i-1][j-1])

# declare objective function
def obj_func(model): 
    return sum(model.x[i,j]*model.d[i,j] for i in model.N for j in model.M)
model.objective = pyEnv.Objective(rule=obj_func, sense=pyEnv.minimize)

# constraint
# creates m constraints defined by rule_const1
# ensures that each building is arrived at from exactly one other building
def rule_const1(model,M):
    return sum(model.x[i,M] for i in model.N if i!=M ) == 1

model.const1 = pyEnv.Constraint(model.M,rule=rule_const1)

# constraint
# creates n constraints defined by rule_const2
# ensures that each building is a departure to exactly one building
def rule_const2(model,N):
    return sum(model.x[N,j] for j in model.M if j!=N) == 1

model.rest2 = pyEnv.Constraint(model.N,rule=rule_const2)

# constraint
# creates txn constraints defined by rule_const3
# subtour elimination
def rule_const3(model,i,j):
    if i!=j: 
       return model.u[i] - model.u[j] + model.x[i,j] * n <= (n-1)
    else:
        #Yeah, this else doesn't say anything
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = pyEnv.Constraint(model.U,model.N,rule=rule_const3)

# no consecutive libraries
def rule_const6(model):
    return model.x[11,14] == 0
model.rest6 = pyEnv.Constraint(rule=rule_const6)

def rule_const7(model):
    return model.x[14,11] == 0
model.rest7 = pyEnv.Constraint(rule=rule_const7)

# Buildings with time constraint go last
def rule_const10(model):
    return model.x[3,4] == 1
model.rest10 = pyEnv.Constraint(rule=rule_const10)

def rule_const11(model):
    return model.x[4,5] == 1
model.rest11 = pyEnv.Constraint(rule=rule_const11)

def rule_const12(model):
    return model.x[5,6] == 1
model.rest12 = pyEnv.Constraint(rule=rule_const12)

def rule_const13(model):
    return model.x[6,1] == 1
model.rest13 = pyEnv.Constraint(rule=rule_const13)

opt = pyEnv.SolverFactory("gurobi", solver_io="python") 
results = opt.solve(model)
results.write()

# prints results
print(results)

# See decision variables
List = list(model.x.keys())
for i in List:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4225.0
  Upper bound: 4225.0
  Number of objectives: 1
  Number of constraints: 386
  Number of variables: 380
  Number of binary variables: 361
  Number of integer variables: 380
  Number of continuous variables: -361
  Number of nonzeros: 1662
  Sense: 1
  Number of solutions: 2
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 9.52
  Status: ok
  Wallclock time: 0.03796958923339844
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an opti